In [5]:

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Configure Ollama LLM
ollama_llm = Ollama(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding

In [9]:
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Context,
)
from llama_index.llms.openai import OpenAI

# Define custom events
class JokeEvent(Event):
    joke: str

class AnalysisRequestEvent(Event):
    joke: str

class AnalysisResponseEvent(Event):
    critique: str


# Define the workflow
class JokeFlow(Workflow):
    llm=ollama_llm
    # llm = OpenAI()

    @step
    async def generate_joke(self, ctx: Context, ev: StartEvent) -> JokeEvent:
        """
        Generate a joke based on the provided topic and store the topic in the global context.
        """
        # Extract the topic from the StartEvent
        topic = ev.topic

        # Store the topic in global context for later use
        await ctx.set("topic", topic)

        # Generate a joke using the LLM
        prompt = f"Write your best joke about {topic}."
        response = await self.llm.acomplete(prompt)

        # Return the generated joke as a JokeEvent
        return JokeEvent(joke=str(response))

    @step
    async def request_analysis(self, ctx: Context, ev: JokeEvent) -> AnalysisRequestEvent:
        """
        Request an analysis of the joke and store the joke in the global context.
        """
        # Extract the joke from the JokeEvent
        joke = ev.joke

        # Store the joke in global context for later use
        await ctx.set("joke", joke)

        # Return an AnalysisRequestEvent to trigger the analysis step
        return AnalysisRequestEvent(joke=joke)

    @step
    async def critique_joke(self, ctx: Context, ev: AnalysisRequestEvent) -> AnalysisResponseEvent:
        """
        Critique the joke and store the critique in the global context.
        """
        # Extract the joke from the AnalysisRequestEvent
        joke = ev.joke

        # Retrieve the joke from the global context
        #joke = await ctx.get("joke")

        # Generate a critique using the LLM
        prompt = f"Give a thorough analysis and critique of the following joke: {joke}"
        response = await self.llm.acomplete(prompt)

        # Store the critique in the global context
        await ctx.set("critique", str(response))

        # Return the critique as an AnalysisResponseEvent
        return AnalysisResponseEvent(critique=str(response))

    @step
    async def finalize_result(
        self, ctx: Context, ev: AnalysisResponseEvent | JokeEvent
    ) -> StopEvent | None:
        """
        Wait for both the JokeEvent and the AnalysisResponseEvent to finalize the result.
        """
        # Wait for both the JokeEvent and AnalysisResponseEvent to arrive
        data = ctx.collect_events(ev, [JokeEvent, AnalysisResponseEvent])

        # If all events are not available yet, return None
        if data is None:
            return None

        # Unpack the collected events
        joke_event, analysis_response_event = data

        # Finalize the result by combining the joke and its critique
        result = f"Joke: {joke_event.joke}\n\nCritique: {analysis_response_event.critique}"

        # Return the finalized result
        return StopEvent(result=result)






In [10]:

# Run the workflow
w = JokeFlow(timeout=120, verbose=True)

async def main():
    # Start the workflow with a topic
    result = await w.run(topic="pirates")

    # Print the final result
    print("\nFinal Result:")
    print(str(result))


In [11]:
await main()

Running step generate_joke
Step generate_joke produced event JokeEvent
Running step finalize_result
Step finalize_result produced no event
Running step request_analysis
Step request_analysis produced event AnalysisRequestEvent
Running step critique_joke
Step critique_joke produced event AnalysisResponseEvent
Running step finalize_result
Step finalize_result produced event StopEvent

Final Result:
Joke: Why did the pirate quit his job?

Because he was sick of all the arrrr-guments! (get it?)

Critique: The joke in question is a play on words, using the nautical term "arrrr" to create a pun. Here's a thorough analysis and critique of the joke:

**Strengths:**

1. **Wordplay**: The use of "arrrr" as a verb, implying disagreement or argument, is a clever play on words. It's a common phrase associated with pirates, but in this context, it takes on a new meaning.
2. **Simple and easy to understand**: The joke is straightforward and doesn't require complex setup or inside knowledge. This make

In [12]:
from llama_index.core.workflow import draw_all_possible_flows

draw_all_possible_flows(w, filename="joke_workflow_context.html")

joke_workflow_context.html


C:\Users\Muthu\AppData\Local\Temp\ipykernel_15160\87609920.py:3: DeprecationWarning: Call to deprecated function (or staticmethod) draw_all_possible_flows. (Install `llama-index-utils-workflow` and use the import `from llama_index.utils.workflow` instead.)
  draw_all_possible_flows(w, filename="joke_workflow_context.html")
